In [2]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 4.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 8.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 20.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier, LGBMRegressor 
# from xgboost import XGBClassifier, XGBRegressor
# from catboost import CatBoostClassifier, CatBoostRegressor

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'March-Mania-2023/MNCAA_train.csv'
file_key_2 = 'March-Mania-2023/MNCAA_test.csv'
file_key_3 = 'March-Mania-2023/WNCAA_train.csv'
file_key_4 = 'March-Mania-2023/WNCAA_test.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

bucket_object_4 = bucket.Object(file_key_4)
file_object_4 = bucket_object_4.get()
file_content_stream_4 = file_object_4.get('Body')

## Reading data files
man_train = pd.read_csv(file_content_stream_1)
man_test = pd.read_csv(file_content_stream_2)
woman_train = pd.read_csv(file_content_stream_3)
woman_test = pd.read_csv(file_content_stream_4)

# Man

In [2]:
woman_test.head()

,ID,Pred,Season,T1,T2,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2
0,2023_3101_3102,0.5,2023,3101,3102,NaN,69.26087,71.0,2.0,57.086957,57.0,43,76,14.086957,2.652174,56.826087,42,0.517398,69.636957,1.740924,0.182232,0.219239,0.667209,NaN,63.680000,63.0,-2.000000,58.480000,57.0,48,67,11.400000,1.640000,52.440000,40,0.448934,71.324000,1.783100,0.199658,0.171136,0.676116,NaN,NaN,NaN,NaN,NaN
1,2023_3101_3103,0.5,2023,3101,3103,NaN,69.26087,71.0,2.0,57.086957,57.0,43,76,14.086957,2.652174,56.826087,42,0.517398,69.636957,1.740924,0.182232,0.219239,0.667209,NaN,66.652174,66.0,0.217391,54.695652,54.0,41,72,15.217391,4.391304,63.347826,48,0.470898,73.357609,1.833940,0.280146,0.216310,0.677596,NaN,NaN,NaN,NaN,NaN
2,2023_3101_3104,0.5,2023,3101,3104,NaN,69.26087,71.0,2.0,57.086957,57.0,43,76,14.086957,2.652174,56.826087,42,0.517398,69.636957,1.740924,0.182232,0.219239,0.667209,NaN,70.592593,69.0,11.925926,57.222222,56.0,44,72,13.333333,4.074074,55.962963,44,0.519303,70.456481,1.761412,0.196778,0.207658,0.674513,NaN,NaN,NaN,NaN,NaN
3,2023_3101_3105,0.5,2023,3101,3105,NaN,69.26087,71.0,2.0,57.086957,57.0,43,76,14.086957,2.652174,56.826087,42,0.517398,69.636957,1.740924,0.182232,0.219239,0.667209,NaN,57.080000,57.0,-4.160000,55.880000,57.0,37,67,9.560000,3.080000,59.160000,44,0.398369,71.142000,1.778550,0.242472,0.234111,0.656382,NaN,NaN,NaN,NaN,NaN
4,2023_3101_3106,0.5,2023,3101,3106,NaN,69.26087,71.0,2.0,57.086957,57.0,43,76,14.086957,2.652174,56.826087,42,0.517398,69.636957,1.740924,0.182232,0.219239,0.667209,NaN,62.120000,65.0,-9.080000,54.840000,55.0,45,70,11.120000,3.600000,61.840000,47,0.434545,73.884000,1.847100,0.273056,0.242199,0.577222,NaN,NaN,NaN,NaN,NaN


In [3]:
woman_train.head()

,Season,DayNum,T1,T2,T1_Points,T2_Points,ResultDiff,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2
0,2010,138,3124,3201,69,55,14,0.5,72.093750,69.0,15.250000,56.343750,56.0,43,70,14.687500,7.250000,62.625000,51,0.484261,72.431250,1.810781,0.318221,0.231303,0.752087,0.750000,73.454545,73.0,12.878788,61.575758,60.0,50,83,14.636364,2.606061,56.696970,43,0.492357,71.998485,1.799962,0.213883,0.217679,0.732616,4,13,-9,1.413986,0.550161
1,2010,138,3173,3395,67,66,1,0.5,70.769231,72.5,11.269231,61.538462,61.0,48,74,14.461538,4.076923,58.692308,42,0.465242,72.958654,1.823966,0.222859,0.231255,0.763468,0.333333,72.300000,71.0,12.000000,62.266667,60.0,46,84,15.600000,3.600000,59.700000,42,0.470597,73.540000,1.838500,0.223105,0.218903,0.685682,8,9,-1,1.227850,0.944480
2,2010,138,3181,3214,72,37,35,1.0,71.593750,70.0,16.531250,63.750000,64.0,46,80,14.250000,5.281250,53.593750,41,0.457585,73.511719,1.837793,0.213418,0.244431,0.739992,1.000000,62.833333,63.5,7.700000,60.100000,58.0,41,80,11.600000,2.233333,49.933333,37,0.412575,69.660000,1.741500,0.231286,0.222436,0.744290,2,15,-13,2.787577,0.250012
3,2010,138,3199,3256,75,61,14,0.0,74.466667,75.5,14.366667,59.200000,58.5,49,76,15.333333,4.033333,59.566667,43,0.499876,74.188333,1.854708,0.269095,0.254897,0.720585,0.800000,74.258065,74.0,9.935484,62.516129,63.0,51,73,14.516129,3.741935,64.064516,52,0.466902,75.900000,1.897500,0.259040,0.224284,0.744287,3,14,-11,1.605326,0.426694
4,2010,138,3207,3265,62,42,20,0.5,68.033333,69.0,9.666667,60.266667,59.0,49,73,15.866667,2.600000,48.866667,31,0.450856,70.078333,1.751958,0.235019,0.231292,0.670805,1.000000,68.181818,70.0,10.272727,55.696970,56.0,47,71,14.454545,4.484848,58.939394,41,0.478543,67.446970,1.686174,0.269434,0.199607,0.689532,5,12,-7,1.464964,1.249223


In [4]:
woman_train['Season'].value_counts()

2022    134
2010    126
2011    126
2012    126
2013    126
2014    126
2015    126
2016    126
2017    126
2018    126
2019    126
2021    126
Name: Season, dtype: int64

In [3]:
man_train['target'] = np.where(man_train['ResultDiff'] > 0, 1, 0)
man_train.head()

,Season,DayNum,T1,T2,T1_Points,T2_Points,ResultDiff,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2,target
0,2003,134,1421,1411,92,84,8,1.000000,71.206897,71.0,-7.241379,56.793103,57.0,45,76,13.034483,3.000000,60.965517,50,0.489800,70.666379,1.766659,0.287716,0.228101,0.697730,0.833333,72.800000,72.5,1.966667,55.266667,55.0,39,73,14.200000,2.233333,60.400000,43,0.503036,70.665000,1.766625,0.330188,0.215143,0.724640,16,16,0,0.866989,0.974984,1
1,2003,136,1112,1436,80,51,29,0.666667,85.214286,86.5,14.964286,65.714286,66.0,50,88,17.642857,4.214286,64.678571,52,0.517632,77.196429,1.929911,0.272214,0.190130,0.724040,1.000000,67.793103,65.0,4.655172,55.862069,58.0,44,67,14.206897,2.965517,55.068966,44,0.494732,66.252586,1.656315,0.235084,0.212863,0.802653,1,16,-15,1.193799,0.975313,1
2,2003,136,1113,1272,84,71,13,0.666667,75.965517,75.0,6.793103,56.896552,57.0,46,69,15.551724,4.241379,55.655172,43,0.517334,69.655172,1.741379,0.313270,0.200649,0.764390,0.750000,74.517241,73.0,8.689655,60.000000,61.0,50,79,16.620690,5.068966,57.862069,42,0.498337,70.600000,1.765000,0.259300,0.194125,0.765180,10,7,3,0.862123,1.049829,1
3,2003,136,1141,1166,79,73,6,1.000000,79.344828,80.0,6.103448,52.689655,53.0,40,63,15.620690,4.000000,57.931034,44,0.572835,72.301724,1.807543,0.369588,0.252913,0.752266,1.000000,79.242424,80.0,14.909091,57.454545,57.0,48,69,16.818182,4.454545,55.333333,45,0.567455,69.453788,1.736345,0.245127,0.192564,0.744973,11,6,5,NaN,1.074422,1
4,2003,136,1143,1301,76,74,2,0.333333,74.482759,75.0,4.724138,58.724138,58.0,42,77,16.000000,2.793103,58.793103,48,0.524098,70.925862,1.773147,0.240967,0.198374,0.736381,0.600000,72.400000,71.5,4.400000,53.333333,52.0,44,73,14.666667,3.066667,53.133333,35,0.534189,67.521667,1.688042,0.298489,0.209940,0.755463,8,9,-1,0.979448,0.894431,1


In [9]:
man_train.tail()

,Season,DayNum,T1,T2,T1_Points,T2_Points,ResultDiff,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2,target
2491,2022,146,1274,1242,50,76,-26,0.75,74.818182,76.0,3.787879,57.636364,57.0,48,70,14.000000,2.939394,57.303030,46,0.542989,68.599242,1.714981,0.218758,0.136049,0.711424,0.833333,78.588235,76.0,10.441176,59.558824,59.0,47,74,15.441176,4.147059,60.029412,48,0.541827,70.923529,1.773088,0.242935,0.169606,0.717460,10,1,9,1.0,1.0,0
2492,2022,146,1389,1314,49,69,-20,1.00,66.172414,66.0,3.931034,54.551724,55.0,46,63,11.965517,4.896552,52.620690,39,0.479555,68.333621,1.708341,0.258885,0.190702,0.756981,0.750000,77.515152,76.0,5.787879,61.030303,60.0,49,81,14.818182,3.848485,62.121212,47,0.522149,71.653788,1.791345,0.232404,0.157818,0.790277,15,8,7,1.0,1.0,0
2493,2022,152,1437,1242,65,81,-16,1.00,72.636364,73.0,9.545455,56.303030,56.0,46,77,12.060606,2.242424,55.515152,45,0.524635,65.116667,1.627917,0.258841,0.143503,0.701769,0.833333,78.588235,76.0,10.441176,59.558824,59.0,47,74,15.441176,4.147059,60.029412,48,0.541827,70.923529,1.773088,0.242935,0.169606,0.717460,2,1,1,1.0,1.0,0
2494,2022,152,1181,1314,77,81,-4,0.60,80.176471,79.5,13.029412,60.764706,61.0,49,79,16.794118,5.441176,62.941176,49,0.556979,69.507353,1.737684,0.213102,0.142988,0.727208,0.750000,77.515152,76.0,5.787879,61.030303,60.0,49,81,14.818182,3.848485,62.121212,47,0.522149,71.653788,1.791345,0.232404,0.157818,0.790277,2,8,-6,1.0,1.0,0
2495,2022,154,1314,1242,69,72,-3,0.75,77.515152,76.0,5.787879,61.030303,60.0,49,81,14.818182,3.848485,62.121212,47,0.522149,71.653788,1.791345,0.232404,0.157818,0.790277,0.833333,78.588235,76.0,10.441176,59.558824,59.0,47,74,15.441176,4.147059,60.029412,48,0.541827,70.923529,1.773088,0.242935,0.169606,0.717460,8,1,7,1.0,1.0,0


In [5]:
man_train.describe()

,Season,DayNum,T1,T2,T1_Points,T2_Points,ResultDiff,X1_WinRatio14d,X1_PointsMean,X1_PointsMedian,X1_PointsDiffMean,X1_FgaMean,X1_FgaMedian,X1_FgaMin,X1_FgaMax,X1_AstMean,X1_BlkMean,X1_OppFgaMean,X1_OppFgaMin,X1_EfgpMean,X1_PossessionsMean,X1_PpmMean,X1_FtrMean,X1_TopMean,X1_DrebpMean,X2_WinRatio14d,X2_PointsMean,X2_PointsMedian,X2_PointsDiffMean,X2_FgaMean,X2_FgaMedian,X2_FgaMin,X2_FgaMax,X2_AstMean,X2_BlkMean,X2_OppFgaMean,X2_OppFgaMin,X2_EfgpMean,X2_PossessionsMean,X2_PpmMean,X2_FtrMean,X2_TopMean,X2_DrebpMean,Seed1,Seed2,SeedDiff,quality_march_T1,quality_march_T2
count,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2496.000000,2462.000000,2462.000000
mean,2012.205929,139.100962,1293.766026,1293.766026,69.438702,69.438702,0.000000,0.724553,74.598962,74.245393,9.078175,57.011400,56.769832,43.774840,72.868189,14.629655,4.040932,56.331005,43.562901,0.524865,68.209404,1.705235,0.269937,0.182848,0.758129,0.724553,74.598962,74.245393,9.078175,57.011400,56.769832,43.774840,72.868189,14.629655,4.040932,56.331005,43.562901,0.524865,68.209404,1.705235,0.269937,0.182848,0.758129,7.035256,7.035256,0.000000,1.085114,1.085114
std,5.635279,4.233399,104.330586,104.330586,12.007556,12.007556,14.411502,0.246563,5.236300,5.354654,4.559931,3.502209,3.600296,4.154118,6.407086,1.854653,1.252531,3.702246,4.156851,0.026685,3.473083,0.086827,0.038709,0.022732,0.035405,0.246563,5.236300,5.354654,4.559931,3.502209,3.600296,4.154118,6.407086,1.854653,1.252531,3.702246,4.156851,0.026685,3.473083,0.086827,0.038709,0.022732,0.035405,4.713809,4.713809,7.499579,0.327834,0.327834
min,2003.000000,134.000000,1101.000000,1101.000000,29.000000,29.000000,-56.000000,0.000000,58.812500,56.500000,-7.241379,42.035714,42.500000,27.000000,52.000000,9.354839,0.655172,40.357143,26.000000,0.419467,54.142857,1.353571,0.167172,0.121706,0.652977,0.000000,58.812500,56.500000,-7.241379,42.035714,42.500000,27.000000,52.000000,9.354839,0.655172,40.357143,26.000000,0.419467,54.142857,1.353571,0.167172,0.121706,0.652977,1.000000,1.000000,-15.000000,0.350650,0.350650
25%,2007.000000,136.000000,1211.000000,1211.000000,61.000000,61.000000,-10.000000,0.600000,71.093040,70.500000,5.937500,54.709677,54.500000,41.000000,69.000000,13.342857,3.151515,53.750000,41.000000,0.506886,65.935813,1.648395,0.243491,0.166931,0.733664,0.600000,71.093040,70.500000,5.937500,54.709677,54.500000,41.000000,69.000000,13.342857,3.151515,53.750000,41.000000,0.506886,65.935813,1.648395,0.243491,0.166931,0.733664,3.000000,3.000000,-5.000000,0.945236,0.945236
50%,2012.000000,137.000000,1284.000000,1284.000000,69.000000,69.000000,0.000000,0.750000,74.375000,74.000000,9.000000,57.000000,57.000000,44.000000,72.000000,14.500000,3.864368,56.171569,43.000000,0.524098,68.071833,1.701796,0.269100,0.181981,0.757756,0.750000,74.375000,74.000000,9.000000,57.000000,57.000000,44.000000,72.000000,14.500000,3.864368,56.171569,43.000000,0.524098,68.071833,1.701796,0.269100,0.181981,0.757756,6.000000,6.000000,0.000000,1.001661,1.001661
75%,2017.000000,139.000000,1393.000000,1393.000000,77.000000,77.000000,10.000000,1.000000,78.096774,78.000000,11.778125,59.258766,59.000000,47.000000,76.000000,15.813920,4.812500,58.694661,46.000000,0.542096,70.575781,1.764395,0.293764,0.198166,0.782150,1.000000,78.096774,78.000000,11.778125,59.258766,59.000000,47.000000,76.000000,15.813920,4.812500,58.694661,46.000000,0.542096,70.575781,1.764395,0.293764,0.198166,0.782150,11.000000,11.000000,5.000000,1.146711,1.146711
max,2022.000000,154.000000,1463.000000,1463.

In [15]:
sum(man_train['Season'] < 2013)

1292

In [4]:
man_train['target'].value_counts()

1    1248
0    1248
Name: target, dtype: int64

In [8]:
man_train.shape

(2496, 49)

In [6]:
man_train['Season'].value_counts()

2012    134
2013    134
2019    134
2018    134
2017    134
2016    134
2015    134
2014    134
2022    134
2011    134
2021    132
2004    128
2010    128
2009    128
2008    128
2007    128
2006    128
2005    128
2003    128
Name: Season, dtype: int64

# Help Funs

In [15]:
def modeling(train, test, model):
    
    X = train.drop(columns = ['Season', 'T1', 'T2', 'T1_Points', 'T2_Points', 'ResultDiff', 'target'], axis = 1)
    Y = train['ResultDiff']
    
    X_test = test.drop(columns = ['Season', 'T1', 'T2', 'T1_Points', 'T2_Points', 'ResultDiff', 'target'], axis = 1)
    Y_test = test['ResultDiff']
    
    if (model == 'XGB'):
        
        XGB_md = XGBClassifier(tree_method = 'hist',
                               colsample_bytree = 0.7, 
                               gamma = 0.8, 
                               learning_rate = 0.01, 
                               max_depth = 7, 
                               min_child_weight = 10, 
                               n_estimators = 1000, 
                               subsample = 0.7).fit(X, Y)
        
        XGB_pred = XGB_md.predict_proba(X_test)[:, 1]
        return mean_squared_error(Y_test, XGB_pred)
        
    if (model == 'LightGBM'):
        
        lgb_md = LGBMClassifier(n_estimators = 1000,
                                max_depth = 7,
                                learning_rate = 0.01,
                                num_leaves = 20,
                                lambda_l1 = 3,
                                lambda_l2 = 3,
                                bagging_fraction = 0.7,
                                feature_fraction = 0.7).fit(X, Y)
        
        lgb_pred = lgb_md.predict_proba(X_test)[:, 1]
        return mean_squared_error(Y_test, lgb_pred)
        
    if (model == 'CatBoost'):
        
        cat_md = CatBoostClassifier(loss_function = 'Logloss',
                                    iterations = 1000,
                                    learning_rate = 0.01,
                                    depth = 7,
                                    random_strength = 0.5,
                                    bagging_temperature = 0.7,
                                    border_count = 30,
                                    l2_leaf_reg = 5,
                                    verbose = False).fit(X, Y)
        
        cat_pred = cat_md.predict_proba(X_test)[:, 1]
        return mean_squared_error(Y_test, cat_pred)
        
    
#     if (model == 'Logistic'):
        
        
#     if (model == 'SVM'):
        
    
#     if (model == 'RF'):
        
    
#     if (model == 'Hist'):
        
    
#     if (model == 'NN'):
        
        
    

# Cross Validation

In [12]:
man_train['Season'].unique()[0:18]

array([2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2021])

In [25]:
seasons = man_train['Season'].unique()
# seasons = seasons[0:]

for i in range(2010, 2022):
    print(i)
    
    train_data = man_train[man_train['Season'] <= i].reset_index(drop = True); print(train_data.shape)
    
    if ((i + 1) == 2020): 
        continue 
    else:
        test_data = man_train[man_train['Season'] == (i + 1)].reset_index(drop = True)
    
    xgb_results = modeling(train_data, test_data, 'XGB')
    print(xgb_results)


2010
(1024, 49)
0.22241378352041136
2011
(1158, 49)
0.20624088946618133
2012
(1292, 49)
0.20404569946608003
2013
(1426, 49)
0.214011961531526
2014
(1560, 49)
0.162914904042113
2015
(1694, 49)
0.20765339701344604
2016
(1828, 49)
0.18527911341205833
2017
(1962, 49)
0.20036769198246995
2018
(2096, 49)
0.18081507893890353
2019
(2230, 49)
2020
(2230, 49)
0.20933848649566694
2021
(2362, 49)
0.25903264537683124
